In [1]:
import numpy as np 

In [2]:
import torch

checkpoint_path = "checkpoint.pth"

# Load the checkpoint with `weights_only=False`
checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=False)

print("Checkpoint loaded successfully!")
print("Available keys:", checkpoint.keys())  # Check contents


Checkpoint loaded successfully!
Available keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'best_val_loss'])


In [3]:

# Load the checkpoint with `weights_only=False`
checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=False)

print("Checkpoint loaded successful/usr/bin/python3 -m pip install ipykernel -U --user --force-reinstallly!")
print("Available keys:", checkpoint.keys())  # Check contents


Checkpoint loaded successful/usr/bin/python3 -m pip install ipykernel -U --user --force-reinstallly!
Available keys: dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'best_val_loss'])


In [8]:
from sklearn.preprocessing import LabelEncoder

# Define the original class labels
class_labels = ["AK", "BCC", "BKL", "DF", "MEL", "NV", "VASC"]

# Initialize and fit LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(class_labels)

# Get the mapping of label names to their encoded numbers
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Reverse the mapping
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Print the mappings
print("Encoded Label -> Original Label Mapping:")
for encoded, original in inverse_label_mapping.items():
    print(f"{encoded} -> {original}")


Encoded Label -> Original Label Mapping:
0 -> AK
1 -> BCC
2 -> BKL
3 -> DF
4 -> MEL
5 -> NV
6 -> VASC


In [9]:
import torch
import timm
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# --------------------------
# 🖥️ Modified ViT Model
# --------------------------
class HybridMHSA(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super(HybridMHSA, self).__init__()
        self.mhsa = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout)
        self.conv = nn.Conv1d(dim, dim, kernel_size=3, padding=1, groups=num_heads)
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.mhsa(x, x, x)
        attn_output = self.dropout(attn_output)
        conv_output = self.conv(x.permute(1, 2, 0))
        conv_output = conv_output.permute(2, 0, 1)
        x = attn_output + conv_output
        x = self.norm(x)
        return x

class ModifiedViT(nn.Module):
    def __init__(self, pretrained_model_name, num_classes, dropout=0.1):
        super(ModifiedViT, self).__init__()
        self.base_model = timm.create_model(pretrained_model_name, pretrained=False, drop_path_rate=dropout)
        self.num_classes = num_classes

        # Replace MHSA with HybridMHSA
        for name, module in self.base_model.named_modules():
            if isinstance(module, nn.MultiheadAttention):
                setattr(
                    self.base_model,
                    name,
                    HybridMHSA(
                        dim=module.embed_dim,
                        num_heads=module.num_heads,
                        dropout=dropout
                    )
                )

        # Update classification head
        self.base_model.head = nn.Linear(self.base_model.head.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# --------------------------
# 🏋️ Load Model from Checkpoint
# --------------------------
def load_model(checkpoint_path, num_classes=7, device="cpu"):
    model = ModifiedViT('vit_base_patch16_224_in21k', num_classes=num_classes, dropout=0.1).to(device)
    checkpoint_pat ="checkpoint.pth"

    checkpoint = torch.load(checkpoint_pat, map_location=device, weights_only=False)

    # Load only the model weights
    model.load_state_dict(checkpoint["model_state_dict"])

    model.eval()  # Set to evaluation mode
    return model

# --------------------------
# 📷 Image Preprocessing
# --------------------------
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)  # Add batch dimension

# --------------------------
# 🔍 Run Inference
# --------------------------
def predict(model, image_path, label_mapping, device="cpu"):
    image = preprocess_image(image_path).to(device)
    with torch.no_grad():
        output = model(image)
        predicted_class = output.argmax(dim=1).item()
    
    # Convert numeric prediction to class name
    predicted_label = label_mapping.get(predicted_class, "Unknown")
    
    return predicted_label

# --------------------------
# 🚀 Main Function
# --------------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = load_model("best_model.pth", device=device)

    # Define label mapping (from your LabelEncoder)
    label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}

    image_path = input("Enter the image path: ").strip()
    predicted_label = predict(model, image_path, label_mapping, device=device)

    print(f"Predicted Class: {predicted_label}")

if __name__ == "__main__":
    main()


/home/som/Desktop/SkinCancer/.venvsom/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/som/Desktop/SkinCancer/.venvsom/lib/python3.13/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


Predicted Class: AK
